# Project Overview

This notebook implements an advanced benchmark of four distinct Vision Language Models (VLMs) on the Hateful Memes Challenge Dataset (HMCD). This version loads the dataset from local files, uses a balanced dataset sample, and evaluates models from four different families to capture classification.

In [ ]:
pip install ollama

In [ ]:
pip install pandas

In [ ]:
pip install scikit-learn


In [185]:
import ollama
import pandas as pd
import os

## Step 1: Load Dataset from Local Files & Prepare Sample

In [199]:
DATASET_FOLDER = 'data' 
ANNOTATION_FILE = os.path.join(DATASET_FOLDER, 'dev.jsonl')
IMG_DIR = os.path.join(DATASET_FOLDER)

In [200]:
IMG_DIR

'data'

In [202]:
if not os.path.exists(ANNOTATION_FILE):
    print(f"Error'")
else:
    df = pd.read_json(ANNOTATION_FILE, lines=True)

    # Create the full path to each image file
    df['img_path'] = df['img'].apply(lambda x: os.path.join(IMG_DIR, x))

    print("Local dataset loaded successfully.")
    print(df['img_path'])


Local dataset loaded successfully.
0      data\img/08291.png
1      data\img/46971.png
2      data\img/03745.png
3      data\img/83745.png
4      data\img/80243.png
              ...        
495    data\img/83675.png
496    data\img/37198.png
497    data\img/48670.png
498    data\img/09863.png
499    data\img/97320.png
Name: img_path, Length: 500, dtype: object


### Balanced Sampling Strategy

In [203]:
df['label_text'] = df['label'].map({0: 'non-hateful', 1: 'hateful'})

In [204]:
hateful_df = df[df['label'] == 1].sample(n=250, random_state=42)

In [205]:
non_hateful_df = df[df['label'] == 0].sample(n=250, random_state=42)

In [206]:
df_sample = pd.concat([hateful_df, non_hateful_df])
df_sample = df_sample.sample(frac=1, random_state=42).reset_index(drop=True)
df_sample['img_path'] = df_sample['img_path'].apply(os.path.normpath) # Normalize all image paths to match your OS

In [207]:
df_sample.head()

,id,img,label,text,img_path,label_text
0,93541,img/93541.png,0,the jihad squad,data\img\93541.png,non-hateful
1,83946,img/83946.png,1,when a white kid shoots up the school the same...,data\img\83946.png,hateful
2,62703,img/62703.png,0,they shot the wrong gorilla,data\img\62703.png,non-hateful
3,85761,img/85761.png,1,no wonder why the pipes are not working,data\img\85761.png,hateful
4,39578,img/39578.png,1,one of the bullets they dug out of a victim fr...,data\img\39578.png,hateful


In [208]:
print(f"Created a balanced test set with {len(df_sample)} samples.")
print("Label distribution:")
print(df_sample['label_text'].value_counts())

Created a balanced test set with 500 samples.
Label distribution:
label_text
non-hateful    250
hateful        250
Name: count, dtype: int64


In [209]:
import os

# Check which files don't exist
missing_files = df_sample[~df_sample['img_path'].apply(os.path.exists)]

if not missing_files.empty:
    print("Missing image files:")
    display(missing_files[['id', 'img_path']])
else:
    print("All image paths are valid.")

All image paths are valid.


## Step 2: Playing with the Prompt

In [210]:
def create_zero_shot_prompt():
    """Asks for a direct classification."""
    return (
        "You are an expert content moderation analyst. Classify the provided meme as either 'hateful' or 'non-hateful'.\n"
        "Respond with only a single word: `hateful` or `non-hateful`."
    )

In [211]:
def create_cot_prompt():
    """Guide the model to reason step-by-step before classifying."""
    return (
        "Perform the following steps:\n"
        "1. Briefly describe the visual elements in the image.\n"
        "2. Analyze the text on the meme.\n"
        "3. Consider the combined meaning of the visual elements and the text.\n\n"
        "Explain your step-by-step analysis and classify the meme as 'hateful' or 'non-hateful'. "
        "Conclude your analysis with a single word classification."
    )


In [212]:
prompt_template = create_zero_shot_prompt()
# prompt_template = create_cot_prompt()

In [213]:
def parse_response(response_text):
    cleaned_text = response_text.lower().strip().replace('.', '')
    if 'non-hateful' in cleaned_text:
        return 'non-hateful'
    elif 'hateful' in cleaned_text:
        return 'hateful'
    else:
        return 'error'

In [214]:
#Test Ollama Server

import requests

try:
    r = requests.get("http://localhost:11434")
    print("Ollama server is running")
except Exception as e:
    print("Ollama is not running:", e)

Ollama server is running


In [144]:
# def test_image_only(image_path):
#     import base64, requests

#     with open(image_path, "rb") as img:
#         image_data = base64.b64encode(img.read()).decode("utf-8")

#     response = requests.post(
#         "http://localhost:11434/api/generate",
#         json={
#             "model": "llava:7b",
#             "prompt": "Describe everything in this image.",
#             "images": [image_data],
#             "stream": False
#         }
#     )
#     print(response.json()["response"])

# test_image_only("data\img\98714.png")

 In the image, a person is seen skydiving from a Cessna aircraft. The individual, clad in a black helmet, has their arms outstretched as if embracing the thrill of the jump. The plane, painted in white and blue, is captured mid-flight against a backdrop of a clear blue sky. On the right side of the image, there's a text that humorously suggests that when the dive instructor starts yelling in Arabic, it's time to start crying for your life. The person appears to be having an enjoyable and adrenaline-filled experience, as indicated by their wide grin. 


In [154]:
# Test for single image

# import base64
# import requests

# model_name = "llava:7b"
# image_path = "data\img\\28017.png"  # Use an actual file path here
# prompt = "What is in this image?"

# def test_llava(model_name, image_path, prompt):
#     try:
#         with open(image_path, "rb") as f:
#             image_data = base64.b64encode(f.read()).decode("utf-8")

#         response = requests.post(
#             "http://localhost:11434/api/generate",
#             json={
#                 "model": model_name,
#                 "prompt": prompt,
#                 "images": [image_data],
#                 "stream": False
#             }
#         )

#         print("Response:", response.json()["response"])
#     except Exception as e:
#         print("Error:", e)

# test_llava(model_name, image_path, prompt_template)

✅ Response:  Non-hateful 


In [215]:
import requests
import base64

def classify_with_ollama(model_name, image_path, prompt):
    try:
        # Read and encode the image
        with open(image_path, "rb") as f:
            image_data = base64.b64encode(f.read()).decode("utf-8")

        # Send HTTP request to Ollama API
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model_name,
                "prompt": prompt,
                "images": [image_data],
                "stream": False
            }
        )

        result = response.json()["response"]

        # You can still use parse_response if needed:
        return parse_response(result)

    except Exception as e:
        return f"error: {e}"

In [107]:
# def classify_with_ollama(model_name, image_path, prompt):
#     try:
#         response = ollama.chat(model=model_name, messages=[{'role': 'user', 'content': prompt, 'images': [image_path]}])
#         return parse_response(response['message']['content'])
#     except Exception as e:
#         return 'error'

In [111]:
# import base64

In [ ]:
# def image_to_base64(image_path):
#     with open(image_path, "rb") as image_file:
#         return base64.b64encode(image_file.read()).decode('utf-8')

In [216]:
from tqdm import tqdm

In [217]:
models_to_test = {
    'llava:7b': classify_with_ollama
}
results_data = {model: [] for model in models_to_test}

for model_name, classification_func in models_to_test.items():
    print(f"\n--- Benchmarking model: {model_name} ---")
    predictions = []
    for index, row in tqdm(df_sample.iterrows(), total=len(df_sample), desc=f"Processing {model_name}"):
        if 'ollama' in classification_func.__name__:
            pred = classification_func(model_name, row['img_path'], prompt_template)
        else:
            pred = classification_func(row['img_path'], prompt_template) ## for gemini or claude
        predictions.append(pred)
    df_sample[f'prediction_{model_name}'] = predictions

print("\n--- Benchmark Complete! ---")
display(df_sample[['id', 'label_text'] + [f'prediction_{model}' for model in models_to_test.keys()]].head())




--- Benchmarking model: llava:7b ---


Processing llava:7b: 100%|██████████| 500/500 [2:33:09<00:00, 18.38s/it]    



--- Benchmark Complete! ---


,id,label_text,prediction_llava:7b
0,93541,non-hateful,hateful
1,83946,hateful,error
2,62703,non-hateful,hateful
3,85761,hateful,hateful
4,39578,hateful,hateful


In [218]:
ground_truth = df_sample['label_text']

In [ ]:
from sklearn.metrics import classification_report

for model_name in models_to_test.keys():
    print(f"\n--- Evaluation Report for: {model_name} ---")
    model_predictions = df_sample[f'prediction_{model_name}']
    report = classification_report(ground_truth, model_predictions, labels=['hateful', 'non-hateful'], zero_division=0)
    print(report)

In [219]:
columns_to_show = ['label_text'] + [col for col in df_sample.columns if 'prediction' in col]
full_results_df = df_sample[columns_to_show]
display(full_results_df)


,label_text,prediction_llava:7b
0,non-hateful,hateful
1,hateful,error
2,non-hateful,hateful
3,hateful,hateful
4,hateful,hateful
...,...,...
495,hateful,non-hateful
496,non-hateful,error
497,non-hateful,non-hateful
498,non-hateful,non-hateful


In [220]:
full_results_df.to_csv('full_benchmark_results.csv', index=False)